In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from utils import *
from dispatch.dispatch import Dispatch, NeuralDispatch
from simulator.base_simulator import BaseSimulator, ManualSimulator
from simulator.simulator import Simulator
from simulator.graphics import plot_CR, plot_counts
from tqdm import tqdm

import torch
from networks.encoders import PointEncoder
from networks.scoring_v1 import ScoringNet, ScoringInterface

from collections import defaultdict
import matplotlib.pyplot as plt
import torch
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from torch import nn
from torchrl.collectors import SyncDataCollector, MultiaSyncDataCollector, MultiSyncDataCollector
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.envs import (
    Compose,
    DoubleToFloat,
    ObservationNorm,
    StepCounter,
    TransformedEnv,
)
from torchrl.envs.libs.gym import GymEnv
from torchrl.envs.utils import check_env_specs, set_exploration_mode
from torchrl.modules import ProbabilisticActor, TanhNormal, ValueOperator
from torchrl.objectives import ClipPPOLoss
from torchrl.objectives.value import GAE
from torchrl.envs import EnvBase
from tensordict.tensordict import TensorDictBase, TensorDict
from typing import Optional
import gym
from torchrl.data import CompositeSpec, BoundedTensorSpec, UnboundedContinuousTensorSpec, BinaryDiscreteTensorSpec, OneHotDiscreteTensorSpec, DiscreteTensorSpec, UnboundedDiscreteTensorSpec

# import importlib
# importlib.reload(utils)

In [25]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

num_cells = 256  # number of cells in each layer
lr = 3e-4
max_grad_norm = 1.0

frame_skip = 1
frames_per_batch = 1000 // frame_skip
# For a complete training, bring the number of frames up to 1M
total_frames = 200_000 // frame_skip

sub_batch_size = 4  # cardinality of the sub-samples gathered from the current data in the inner loop
num_epochs = 10  # optimisation steps per batch of data collected
clip_epsilon = (
    0.2  # clip value for PPO loss: see the equation in the intro for more context.
)
gamma = 0.99
lmbda = 0.95
entropy_eps = 1e-4

cpu


In [26]:
net = ScoringNet(
    mode='default',
    # point_encoder=point_encoder,
    n_layers=2,
    d_model=512,
    n_head=4,
    dim_ff=512,
    point_enc_dim=64,
    number_enc_dim=8,
    device=device
)

model = ScoringInterface(net)
# model.load_weights('pretrained_models/assignment_cloning_model_v2')

In [27]:
bounds = (Point(0, 0), Point(10, 10))
triples = [random_triple(bounds) for _ in range(sub_batch_size)]

model.encode_input(triples, 0)
preds = model.inference()
mask = model.get_mask()

print(preds.shape, mask.shape)

torch.Size([4, 9, 12]) torch.Size([4, 9, 11])


In [28]:
model.encode_input(triples[0], 0)
model.get_mask().shape

torch.Size([3, 5])

In [29]:
optimizer = torch.optim.SGD(model.net.parameters(), lr=1e-4, momentum=0.9) # FIX unfreeze encoder params
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, epochs=n_epochs, steps_per_epoch=n_iters)
scheduler = None

losses = []
for epoch in range(2):
    for iter in range(10):
        rolling_loss = []
        triples = [random_triple(bounds) for _ in range(sub_batch_size)]

        scorer = ETAScoring()
        solver = HungarianSolver()
        assignments = []
        for triple in triples:
            scores = scorer(triple.orders, triple.couriers)
            assignments.append([(i, j) for i, j in zip(*solver(scores))])

        # print(assignments)
        # print(50 * '-')

        optimizer.zero_grad()
        model.encode_input(triples, 0)
        model.inference()
        loss = model.CE_loss(assignments)
        loss.backward()
        optimizer.step()
        rolling_loss.append(loss.item())

    losses.append(np.mean(rolling_loss))

    print(losses)

[23.777982711791992]
[23.777982711791992, 11.678504943847656]


In [30]:
# dsp = NeuralDispatch(net)
dsp = Dispatch()
sim = Simulator()

all_metrics = []
for i in tqdm(range(1000)):
    metrics = sim.GetMetrics()
    all_metrics.append(metrics)
    state = sim.GetState()
    sim.Next(dsp(state))

plot_CR(all_metrics)
plot_counts(all_metrics)

FileNotFoundError: [Errno 2] No such file or directory: 'environment_config.json'

In [8]:
# # dsp = NeuralDispatch(net)
# dsp = Dispatch()
# sim = ManualSimulator()

# all_metrics = []
# for i in tqdm(range(1000)):
#     metrics = sim.GetMetrics()
#     all_metrics.append(metrics)
#     ass = dsp(sim.GetState())
#     sim.Next(ass)

# plot_CR(all_metrics)
# plot_counts(all_metrics)

In [9]:
# model.tensors['o'].shape
model.masks['o'].shape

torch.Size([4, 10])

In [17]:
MAX_COURIERS = 50

class SimulatorEnv(EnvBase):
    def __init__(self, simulator: type[Simulator], seed=None, device="cpu"):
        super().__init__(device=device, batch_size=[])
        # self.simulators = [simulator(seed=i) for i in range(sub_batch_size)]
        self.simulator = simulator()
        self.model = ScoringInterface(net)
        # triples = [simulator.GetState() for simulator in self.simulators]
        # self.model.encode_input(triples, 0)
        triple = self.simulator.GetState()
        self.model.encode_input(triple, 0)
        self._make_specs()
        
        if seed is None:
            seed = torch.empty((), dtype=torch.long).random_().item()
        self.set_seed(seed)
        
    def _step(self, tensordict: TensorDictBase) -> TensorDictBase:
        '''
        tensordict['action'] - a np.array of indexes of couriers assigned for the given order. If there is no courier assigned -1 is provided.
        BOS-fake items are included.
        '''

        # BATCH MODE
        # for batch_idx in range(sub_batch_size):
        #     assignments = []
        #     assigned_o_idxs = set()
        #     assigned_c_idxs = set()
        #     for o_idx, c_idx in enumerate(tensordict['action'][batch_idx].numpy()):
        #         if c_idx != - 1 \
        #             and not tensordict['observation', 'masks', 'o'][batch_idx][o_idx] \
        #             and not tensordict['observation', 'masks', 'c'][batch_idx][c_idx] \
        #             and (o_idx not in assigned_o_idxs) and (c_idx not in assigned_c_idxs) \
        #         :
        #             assignment = (tensordict['observation', 'ids', 'o'][batch_idx][o_idx].item(), tensordict['observation', 'ids', 'c'][batch_idx][c_idx].item())
        #             assignments.append(assignment)
        #             assigned_o_idxs.add(o_idx)
        #             assigned_c_idxs.add(c_idx)

        #     self.simulators[batch_idx].Next(assignments)

        # print(tensordict.flatten_keys())
        # print(tensordict['observation', 'masks', 'o'])
        # print(tensordict['observation', 'ids', 'o'])
        # print(tensordict['observation', 'masks', 'c'])
        # print(tensordict['observation', 'ids', 'c'])
        # print(tensordict['action'])

        assignments = []
        assigned_o_idxs = set()
        assigned_c_idxs = set()
        for o_idx, c_idx in enumerate(tensordict['action'].numpy()):
            if c_idx >= self.model.ids['c'].shape[-1]:
                print('WARNING!\n\nc_idx increases MAX_LIMIT')
                continue
            if c_idx != - 1 \
                and not tensordict['observation', 'masks', 'o'][o_idx] \
                and not tensordict['observation', 'masks', 'c'][c_idx] \
                and (o_idx not in assigned_o_idxs) and (c_idx not in assigned_c_idxs) \
            :
                assignment = (tensordict['observation', 'ids', 'o'][o_idx].item(), tensordict['observation', 'ids', 'c'][c_idx].item())
                assignments.append(assignment)
                assigned_o_idxs.add(o_idx)
                assigned_c_idxs.add(c_idx)

        # print(assignments)
        # print(self.simulator.GetState())
        self.simulator.Next(assignments)
        triple = self.simulator.GetState()
        self.model.encode_input(triple, 0)

        out = TensorDict(
            {
                "next": {
                    "observation": {
                        'tensors': {
                            'o': self.model.tensors['o'],
                            'c': self.model.tensors['c'],
                            'ar': self.model.tensors['ar']     
                        },
                        'masks': {
                            'o': self.model.masks['o'],
                            'c': self.model.masks['c'],
                            'ar': self.model.masks['ar']     
                        },
                        'ids': {
                            'o': self.model.ids['o'],
                            'c': self.model.ids['c'],
                            'ar': self.model.ids['ar']
                        }
                    },
                    "reward": torch.tensor(0, dtype=torch.float32),
                    "done": torch.tensor(False, dtype=torch.bool),
                    # "reward": torch.tensor([0] * sub_batch_size, dtype=torch.float32),
                    # "done": torch.tensor([False] * sub_batch_size, dtype=torch.bool),
                }
            },
            tensordict.shape
            # batch_size=tensordict.shape[0]
        )
        # print(out['next', 'observation', 'ids', 'o'])
        return out
    
    def _reset(self, tensordict: TensorDictBase) -> TensorDictBase:
        if tensordict is None or tensordict.is_empty():
            return TensorDict(
            {
                "observation": {
                    'tensors': {
                        'o': self.model.tensors['o'],
                        'c': self.model.tensors['c'],
                        'ar': self.model.tensors['ar']     
                    },
                    'masks': {
                        'o': self.model.masks['o'],
                        'c': self.model.masks['c'],
                        'ar': self.model.masks['ar']     
                    },
                    'ids': {
                        'o': self.model.ids['o'],
                        'c': self.model.ids['c'],
                        'ar': self.model.ids['ar']
                    }
                }
            },
            # batch_size=[sub_batch_size]
            batch_size=self.batch_size
        ) 
        return tensordict
    
    def _set_seed(self, seed: Optional[int]):
        rng = torch.manual_seed(seed)
        self.rng = rng

    def _make_specs(self) -> None:
        self.action_spec = DiscreteTensorSpec(
            n=MAX_COURIERS,
            dtype=torch.int,
            shape=self.model.masks['o'].shape
        )
        observation_spec = CompositeSpec(
            tensors = CompositeSpec(
                o = UnboundedContinuousTensorSpec(
                    shape=self.model.tensors['o'].shape,
                    dtype=torch.float
                ),
                c = UnboundedContinuousTensorSpec(
                    shape=self.model.tensors['c'].shape,
                    dtype=torch.float
                ),
                ar = UnboundedContinuousTensorSpec(
                    shape=self.model.tensors['ar'].shape,
                    dtype=torch.float
                ),
                # shape=[sub_batch_size]
            ),
            masks = CompositeSpec(
                o = DiscreteTensorSpec(
                    n=2,
                    dtype=torch.bool,
                    shape=self.model.masks['o'].shape
                ),
                c = DiscreteTensorSpec(
                    n=2,
                    dtype=torch.bool,
                    shape=self.model.masks['c'].shape
                ),
                ar = DiscreteTensorSpec(
                    n=2,
                    dtype=torch.bool,
                    shape=self.model.masks['ar'].shape
                ),
                # shape=[sub_batch_size]
            ),
            ids = CompositeSpec(
                o = UnboundedDiscreteTensorSpec(
                    dtype=torch.int,
                    shape=self.model.ids['o'].shape
                ),
                c = UnboundedDiscreteTensorSpec(
                    dtype=torch.int,
                    shape=self.model.ids['c'].shape
                ),
                ar = UnboundedDiscreteTensorSpec(
                    dtype=torch.int,
                    shape=self.model.ids['ar'].shape
                ),
                # shape=[sub_batch_size]
            ), 
            # shape=[sub_batch_size]
        )
        # if not isinstance(observation_spec, CompositeSpec):
        observation_spec = CompositeSpec(observation=observation_spec) # shape=[sub_batch_size]
            
        self.observation_spec = observation_spec
        self.reward_spec = UnboundedContinuousTensorSpec(
            # shape=[sub_batch_size],
            shape=[1],
            dtype=torch.float32,
        )
        self.done_spec = BinaryDiscreteTensorSpec(
            # n=sub_batch_size,
            # shape=[sub_batch_size],
            n=1,
            shape=[1],
            dtype=torch.bool
        )
 

In [18]:
my_env = SimulatorEnv(Simulator)

In [19]:
env = TransformedEnv(
    my_env,
    # Compose(
    #     # normalize observations
    #     # ObservationNorm(in_keys=["observation"]),
    #     # DoubleToFloat(in_keys=["observation"]),
    #     StepCounter(),
    # ),
)

In [20]:
# env.transform[0].init_stats(num_iter=1000, reduce_dim=0, cat_dim=0)
# print("normalization constant shape:", env.transform[0].loc.shape)
print("observation_spec:", my_env.observation_spec)
print("reward_spec:", my_env.reward_spec)
print("input_spec:", my_env.input_spec)
print("action_spec (as defined by input_spec):", my_env.action_spec)

observation_spec: CompositeSpec(
    observation: CompositeSpec(
        tensors: CompositeSpec(
            o: UnboundedContinuousTensorSpec(
                 shape=torch.Size([4, 512]), space=None, device=cpu, dtype=torch.float32, domain=continuous),
            c: UnboundedContinuousTensorSpec(
                 shape=torch.Size([21, 512]), space=None, device=cpu, dtype=torch.float32, domain=continuous),
            ar: UnboundedContinuousTensorSpec(
                 shape=torch.Size([1, 512]), space=None, device=cpu, dtype=torch.float32, domain=continuous), device=cpu, shape=torch.Size([])),
        masks: CompositeSpec(
            o: DiscreteTensorSpec(
                 shape=torch.Size([4]), space=DiscreteBox(n=2), device=cpu, dtype=torch.bool, domain=discrete),
            c: DiscreteTensorSpec(
                 shape=torch.Size([21]), space=DiscreteBox(n=2), device=cpu, dtype=torch.bool, domain=discrete),
            ar: DiscreteTensorSpec(
                 shape=torch.Size([1]

In [21]:
# check_env_specs(my_env, return_contiguous=False)

In [22]:
rollout = my_env.rollout(10, return_contiguous=False)
# print("rollout of three steps:", rollout)
print("Shape of the rollout TensorDict:", rollout.batch_size)
rollout.batch_size

WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MAX_LIMIT
WARNING!

c_idx increases MA

torch.Size([10])

In [23]:
rollout

RuntimeError: Found more than one unique shape in the tensors to be stacked ({torch.Size([6]), torch.Size([8]), torch.Size([4]), torch.Size([7]), torch.Size([10]), torch.Size([3]), torch.Size([9])}). This is likely due to a modification of one of the stacked TensorDicts, where a key has been updated/created with an uncompatible shape. If the entries are intended to have a different shape, use the get_nestedtensor method instead.

In [26]:
module = TensorDictModule(
    net, in_keys=[('observation', 'tensors'), ('observation', 'masks')], out_keys=['logits', 'values']
)

In [17]:
# td = my_env.reset()
# net(td['observation', 'tensors'].to_dict(), td['observation', 'masks'].to_dict())
# td['observation', 'tensors'].to_dict()

In [27]:
print("Running policy:", module(my_env.reset()).shape)
# env.reset()

Running policy: torch.Size([])


In [22]:
# for i in [0, 1, 2, 3]:
#     print(my_env.simulators[i].GetMetrics())
#     print('-'* 50)


In [28]:
# from tensordict.nn.distributions import Categorical
from torch.distributions.categorical import Categorical
policy_module_actor = ProbabilisticActor(
    module=module,
    in_keys=["logits"],
    distribution_class=Categorical,
    # distribution_kwargs={
    #     "n": env.action_spec.space.n,
    # },
    return_log_prob=True,
    # we'll need the log-prob for the numerator of the importance weights
)

In [69]:
class LogPolicyAvg(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input, mask):
        assert (len(input.shape) == 2 and len(mask.shape) == 2) or (len(input.shape) == 1 and len(mask.shape) == 1), 'dims should be [bs, ord] or [ord]'

        sums = torch.sum(torch.where(mask, input, 0), dim=-1)
        nums = torch.sum(torch.where(mask, 1, 0), dim=-1)

        return torch.where(nums > 0, sums / nums, 0)

In [70]:
from tensordict.nn import TensorDictModule, TensorDictSequential
final_module = TensorDictSequential(
    policy_module_actor,
    TensorDictModule(LogPolicyAvg(), in_keys=['sample_log_prob', ('observation', 'masks', 'o')], out_keys=['sample_log_prob']),
)


In [40]:
# value_module = ValueOperator(
#     module=net,
#     in_keys=[('observation', 'tensors'), ('observation', 'masks')], 
#     out_keys=['logits', 'values']
# )

In [41]:
collector = SyncDataCollector(
    my_env,
    policy_module_actor,
    frames_per_batch=1,
    total_frames=5,
    split_trajs=False,
    device=device,
)

In [42]:
for i, c in enumerate(collector):
    if i == 1:
        break
c

RuntimeError: The expanded size of the tensor (4) must match the existing size (18) at non-singleton dimension 0.  Target sizes: [4, 512].  Tensor sizes: [18, 512]

In [76]:
from torchrl.data import ReplayBuffer, LazyTensorStorage
replay_buffer = ReplayBuffer(
    storage=LazyTensorStorage(frames_per_batch),
    sampler=SamplerWithoutReplacement(),
    batch_size=sub_batch_size
)

In [77]:
advantage_module = GAE(
    gamma=gamma, lmbda=lmbda, value_network=module, average_gae=True, value_key='values'
)

loss_module = ClipPPOLoss(
    actor=policy_module_actor,
    critic=final_module,
    advantage_key="advantage",
    clip_epsilon=clip_epsilon,
    entropy_bonus=bool(entropy_eps),
    entropy_coef=entropy_eps,
    # these keys match by default but we set this for completeness
    value_target_key=advantage_module.value_target_key,
    critic_coef=1.0,
    gamma=0.99,
    loss_critic_type="smooth_l1",
)

optim = torch.optim.Adam(loss_module.parameters(), lr=3e-4)

In [78]:
for i, c in enumerate(collector):
    if i == 5:
        break
    advantage_module(c)
    data_view = c.reshape(-1)
    replay_buffer.extend(data_view.cpu())

subdata = replay_buffer.sample()
# loss_vals = loss_module(subdata)
# loss_value = (
#     loss_vals["loss_objective"]
#     + loss_vals["loss_critic"]
#     + loss_vals["loss_entropy"]
# )


In [79]:
subdata.flatten_keys()

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([4, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        advantage: Tensor(shape=torch.Size([4, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        collector.traj_ids: Tensor(shape=torch.Size([4]), device=cpu, dtype=torch.int64, is_shared=False),
        done: Tensor(shape=torch.Size([4, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        logits: Tensor(shape=torch.Size([4, 1, 22]), device=cpu, dtype=torch.float32, is_shared=False),
        next.done: Tensor(shape=torch.Size([4, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next.observation.ids.ar: Tensor(shape=torch.Size([4, 1]), device=cpu, dtype=torch.int32, is_shared=False),
        next.observation.ids.c: Tensor(shape=torch.Size([4, 21]), device=cpu, dtype=torch.int32, is_shared=False),
        next.observation.ids.o: Tensor(shape=torch.Size([4, 1]), device=cpu, dtype=torch.int32, is_shared=False),
        next.observation

In [80]:
loss_vals = loss_module(subdata)

RuntimeError: advantage.shape and log_weight.shape do not match (got torch.Size([4, 1]) and torch.Size([4, 1, 1]))

In [ ]:
pbar = tqdm(total=total_frames * frame_skip)
eval_str = ""

# We iterate over the collector until it reaches the total number of frames it was
# designed to collect:
for i, tensordict_data in enumerate(collector):
    # we now have a batch of data to work with. Let's learn something from it.
    for _ in range(num_epochs):
        # We'll need an "advantage" signal to make PPO work.
        # We re-compute it at each epoch as its value depends on the value
        # network which is updated in the inner loop.
        advantage_module(tensordict_data)
        data_view = tensordict_data.reshape(-1)
        replay_buffer.extend(data_view.cpu())
        for _ in range(frames_per_batch // sub_batch_size):
            subdata = replay_buffer.sample(sub_batch_size)
            loss_vals = loss_module(subdata.to(device))
            loss_value = (
                loss_vals["loss_objective"]
                + loss_vals["loss_critic"]
                + loss_vals["loss_entropy"]
            )

            # Optimization: backward, grad clipping and optim step
            loss_value.backward()
            # this is not strictly mandatory but it's good practice to keep
            # your gradient norm bounded
            torch.nn.utils.clip_grad_norm_(loss_module.parameters(), max_grad_norm)
            optim.step()
            optim.zero_grad()
    break

## Testing area

In [19]:
bs = 2

class TestEnv(EnvBase):
    def __init__(self, seed=None, device="cpu"):
        super().__init__(device=device, batch_size=[bs])
        self._make_specs()
        
        if seed is None:
            seed = torch.empty((), dtype=torch.long).random_().item()
        self.set_seed(seed)
        
    def _step(self, tensordict: TensorDictBase) -> TensorDictBase:
        '''
        tensordict['action'] - a np.array of indexes of couriers assigned for the given order. If there is no courier assigned -1 is provided.
        '''
        print('next')
        out = TensorDict(
            {
                "next": {
                    "observation": TensorDict({
                        'a': torch.tensor([[-1, -2], [-4, -5]], dtype=torch.float),
                        'b': torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float),
                        },
                        batch_size=[bs]
                    ),
                    "reward": torch.tensor([0] * bs, dtype=torch.float32),
                    "done": torch.tensor([False] * bs, dtype=torch.bool),
                }
            },
            batch_size=[bs]
        )
        return out
    
    def _reset(self, tensordict: TensorDictBase) -> TensorDictBase:
        if tensordict is None or tensordict.is_empty():
            return TensorDict(
            {
                "observation": TensorDict({
                        'a': torch.tensor([[-1, -2], [-4, -5]], dtype=torch.float),
                        'b': torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float),
                        },
                        batch_size=[bs]
                    ),
            },
            batch_size=[bs]
        ) 
        return tensordict
    
    def _set_seed(self, seed: Optional[int]):
        rng = torch.manual_seed(seed)
        self.rng = rng

    def _make_specs(self) -> None:
        self.action_spec = UnboundedDiscreteTensorSpec(
            dtype=torch.int,
            shape=[bs]
            # shape=(1,)
            # shape=(self.model.tensors['o'].shape[0],)
        )
        observation_spec = CompositeSpec(
            a = UnboundedContinuousTensorSpec(
                dtype=torch.float32,
                shape=[bs]
            ),
            b = UnboundedContinuousTensorSpec(
                dtype=torch.float32,
                shape=[bs]
            ),
            shape=[bs]
        )
        # if not isinstance(observation_spec, CompositeSpec):
        observation_spec = CompositeSpec(observation=observation_spec, shape=[bs])
            
        self.observation_spec = observation_spec
        self.reward_spec = UnboundedContinuousTensorSpec(
            shape=[bs],
            dtype=torch.float32,
        )
        self.done_spec = BinaryDiscreteTensorSpec(
            bs,
            shape=[bs]
        )
 

In [20]:
env = TransformedEnv(
    TestEnv(),
    # Compose(
    #     # normalize observations
    #     # ObservationNorm(in_keys=["observation"]),
    #     # DoubleToFloat(in_keys=["observation"]),
    #     StepCounter(),
    # ),
)

In [21]:
print("observation_spec:", env.observation_spec)
print("reward_spec:", env.reward_spec)
print("input_spec:", env.input_spec)
print("action_spec (as defined by input_spec):", env.action_spec)

check_env_specs(env)

observation_spec: CompositeSpec(
    observation: CompositeSpec(
        a: UnboundedContinuousTensorSpec(
             shape=torch.Size([2, 3]), space=None, device=cpu, dtype=torch.float32, domain=continuous),
        b: UnboundedContinuousTensorSpec(
             shape=torch.Size([2, 3]), space=None, device=cpu, dtype=torch.float32, domain=continuous), device=cpu, shape=torch.Size([2, 3])), device=cpu, shape=torch.Size([2]))
reward_spec: UnboundedContinuousTensorSpec(
     shape=torch.Size([2]), space=None, device=cpu, dtype=torch.float32, domain=continuous)
input_spec: CompositeSpec(
    action: UnboundedDiscreteTensorSpec(
         shape=torch.Size([2]), space=ContinuousBox(minimum=Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.int64, contiguous=True), maximum=Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.int64, contiguous=True)), device=cpu, dtype=torch.int32, domain=continuous), device=cpu, shape=torch.Size([2]))
action_spec (as defined by input_spec): UnboundedDis

AssertionError: The dtypes of the real and fake tensordict don't match for key next.done. Got fake=torch.int64 and real=torch.bool.

In [89]:
fake = env.fake_tensordict().flatten_keys(".")
print(fake)

real_tensordict = env.rollout(3, return_contiguous=True).flatten_keys(".")
print(real_tensordict)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.int32, is_shared=False),
        done: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.int64, is_shared=False),
        next.done: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.int64, is_shared=False),
        next.observation.a: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        next.observation.b: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        next.reward: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        observation.a: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        observation.b: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([2]),
    device=cpu,
    is_shared=F

In [91]:
fake_tensordict = fake.unsqueeze(real_tensordict.batch_dims - 1).expand(*real_tensordict.shape).to_tensordict()

In [93]:
# real_tensordict.apply(lambda x: torch.zeros_like(x)) == fake_tensordict.apply(lambda x: torch.zeros_like(x))
# fake_tensordict
fake_tensordict

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.int32, is_shared=False),
        done: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.int64, is_shared=False),
        next.done: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.int64, is_shared=False),
        next.observation.a: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        next.observation.b: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        next.reward: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        observation.a: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        observation.b: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([2, 3]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([2, 3]),
 

In [131]:
class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.f1 = nn.Linear(3, 5)
        self.f2 = nn.Linear(3, 5)

        self.last = nn.Linear(5, 2)
    
    def forward(self, x, y):
        x = self.f1(x)
        y = self.f2(y)

        return self.last(x - y)

In [137]:
x = torch.tensor([[1, 2, 3]], dtype=torch.float)
y = torch.tensor([[1, 2, 3]], dtype=torch.float)
n = Net()
n(x, y)

tensor([[-1.7460, -0.8222]], grad_fn=<AddmmBackward0>)

In [138]:
policy_module = TensorDictModule(
    n, in_keys=[('inp', "x"), ('inp', "y")], out_keys=["out"]
)

In [146]:
# policy_module(x=x, y=y)
inp = TensorDict(
    {
        'inp': TensorDict({
            'x': x,
            'y': y,
        }, batch_size=())
    },
    batch_size=(1, 3)
)

In [140]:
out = policy_module(inp)

In [23]:
out['out']

tensor([1.9481, 0.0790], grad_fn=<AddBackward0>)

In [147]:
inp.shape

torch.Size([1, 3])